In [1]:
from SigProfilerAssignment import Analyzer as Analyze
import sigProfilerPlotting as sigPlt
import os
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt 
import seaborn as sns

# COSMIC DATABASE (SigProfilerAssignment)

In [2]:
input_data = '/data/project/Alzheimer/Main/00.Neurogenesis/04.analysis/sigprofiler_Assignment/SGZ/vcf/'
output = '/data/project/Alzheimer/Main/00.Neurogenesis/04.analysis/sigprofiler_Assignment/SGZ/result/'

input_data = '/home/goldpm1/Meningioma/04.mutect/06.signature/'
output = '/home/goldpm1/Meningioma/04.mutect/07.signature_result/'

if not os.path.exists(output):
        os.makedirs(output)

Analyze.cosmic_fit(input_data, output, input_type = "vcf", genome_build="GRCh38", context_type = "96", exome =True,
                   signatures = None, signature_database = None, collapse_to_SBS96=False, make_plots = True,
                   exclude_signature_subgroups = ["Chemotherapy_signatures", "Treatment_signatures", "Artifact_signatures", "Test2"] )

Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 3.06 seconds.
Starting matrix generation for INDELs...Completed! Elapsed time: 3.14 seconds.
Matrices generated for 6 samples with 0 errors. Total of 140 SNVs, 1 DINUCs, and 11 INDELs were successfully analyzed.
Removing samples with zero TMB ...... 
The following signatures are excluded: SBS11 SBS25 SBS31 SBS35 SBS86 SBS87 SBS90 SBS11 SBS25 SBS31 SBS32 SBS35 SBS86 SBS87 SBS90 SBS27 SBS43 SBS45 SBS46 SBS47 SBS48 SBS49 SBS50 SBS51 SBS52 SBS53 SBS54 SBS55 SBS56 SBS57 SBS58 SBS59 SBS60 SBS95 SBS42
Assigning COSMIC sigs or Signature Database ...... 
|████████████████████████████████████████| 5/5 [100%] in 0.5s (10.06/s) 


 
Your Job Is Successfully Completed! Thank You For Using SigProfilerAssignment.
 


## SBS signature fitting stack bar

In [3]:
f = open(output + 'Assignment_Solution/Activities/Assignment_Solution_Activities.txt','r')

data = f.readlines()

column_name = data[0].rstrip().split('\t')
data_split = [x.rstrip().split('\t') for x in data[1:]]

df = pd.DataFrame(data_split, columns = column_name)
df = df.set_index(keys = 'Samples')

for i in df.columns:
    df[i] = pd.to_numeric(df[i])

### 0으로 가득찬 df는 filter하기

In [4]:
for column in df.columns:
    check = set()
    for value in df[column]:
        check.add(value)
    if len(check) == 1:
        df = df.drop(column, axis = 'columns')

df

,SBS5,SBS15,SBS16,SBS23,SBS38,SBS39,SBS44,SBS84
Samples,,,,,,,,
220930_Dura,0,5,0,4,0,0,0,0
220930_multiple,25,0,8,0,0,0,0,0
221026_Dura,32,0,0,0,0,0,13,20
221026_multiple,6,0,0,0,4,9,0,0
221102_multiple,0,5,0,4,0,0,0,0


### Percentage 로 변환하기

In [5]:
SBS_name = df.columns
total_count = {}

for index, row in df.iterrows():  ## index == A.~~ row = value
    total = 0
    for value in row:
        total += value
    total_count[index] = total

for index, row in df.iterrows():
    for idx, value in enumerate(row):
        df[SBS_name[idx]][index] = value/total_count[index]*100
        
df

,SBS5,SBS15,SBS16,SBS23,SBS38,SBS39,SBS44,SBS84
Samples,,,,,,,,
220930_Dura,0,55,0,44,0,0,0,0
220930_multiple,75,0,24,0,0,0,0,0
221026_Dura,49,0,0,0,0,0,20,30
221026_multiple,31,0,0,0,21,47,0,0
221102_multiple,0,55,0,44,0,0,0,0


In [44]:
fig, ax = plt.subplots( figsize=(12, 6), nrows = 1, ncols = 1 )
#plt.figure ( figsize = (20, 15) )
sns.set_style("white")
ax.set_title ("Stacked bar chart", fontsize = 24, fontweight='heavy')
df.plot (kind = "bar", stacked=True , ax = ax)
ax.set_xticklabels (list ( df.index ), rotation  = 0, fontsize = 12, fontweight='bold')
ax.set_xlabel("")
plt.savefig ("/home/goldpm1/Meningioma/script/41.SigProfiler/barchart.jpg", dpi = 300)
plt.show()

# DE NOVO SIGNATURE (SigProfilerExtractor)

In [47]:
from SigProfilerExtractor import sigpro as sig

sig.sigProfilerExtractor (input_type = "vcf", output = output, input_data = input_data, reference_genome="GRCh38", context_type = "96,DINUC,ID", exome =True,
                                    minimum_signatures=1, maximum_signatures=10, nmf_replicates=100, resample = True, batch_size=1, cpu=-1, gpu=False, 
                                    nmf_init="random", precision= "single", matrix_normalization= "gmm", seeds = "random", 
                                    min_nmf_iterations= 10000, max_nmf_iterations=1000000, nmf_test_conv= 10000, nmf_tolerance= 1e-15, get_all_signature_matrices= False,
                                    make_decomposition_plots = True)
                   #signatures = None, signature_database = None, collapse_to_SBS96=False, make_plots = True )


************** Reported Current Memory Use: 1.97 GB *****************

Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 1.6 seconds.
Starting matrix generation for INDELs...Completed! Elapsed time: 1.74 seconds.
Matrices generated for 6 samples with 0 errors. Total of 140 SNVs, 1 DINUCs, and 11 INDELs were successfully analyzed.
Extracting signature 1 for mutation type 96
The matrix normalizing cutoff is 9600


Time taken to collect 100 iterations for 1 signatures is 14.7 seconds
Optimization time is 4.684453010559082 seconds
The reconstruction error is 0.5627, average process stability is 1.0 and 
the minimum process stability is 1.0 for 1 signatures


Extracting signature 2 for mutation type 96
The matrix normalizing cutoff is 9600


Time taken to collect 100 iterations for 2 signatures is 17.91 seconds
Optimization time is 4.435749769210815 seconds
The reconstruction error is 0.4565, average process stability is 0.33 and 
the minimum process stability is 0.

KeyError: 'make_decomposition_plots'